In [1]:
import tensorflow as tf
import numpy as np
import gradio as gr
from skimage.transform import resize

In [2]:
# Load saved model from model.tflite file
interpreter = tf.lite.Interpreter(model_path="../model.tflite")
interpreter.allocate_tensors()

print(interpreter.get_input_details())
print(interpreter.get_output_details())

[{'name': 'serving_default_conv2d_6_input:0', 'index': 0, 'shape': array([  1, 224, 224,   3]), 'shape_signature': array([ -1, 224, 224,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 25, 'shape': array([1, 4]), 'shape_signature': array([-1,  4]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [3]:
# Classifier
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

def classify_img(img):
    # preprocessing: resizing to appropriate shape, making pixel values between 0 and 1
    resized = resize(img, (224, 224))
    new_arr = resized.astype(np.float32)

    interpreter.set_tensor(input_details[0]['index'], [new_arr])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    results = {}
    results["Heavy damage"] = output_data[0][0]
    results["Minor damage"] = output_data[0][1]
    results["Moderate damage"] = output_data[0][2]
    results["Undamaged"] = output_data[0][3]
    return results

In [4]:
# Create demo with Gradio
demo = gr.Interface(fn=classify_img, inputs=gr.Image(type="numpy"), outputs="label")
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
